# Binary Prediction of Poisonous Mushrooms

### Artificial Intelligence 2nd Project

The aim of this project is to implement and evaluate machine learning models for predicting whether a mushroom is **poisonous** or **edible** based on its physical characteristics. As such, this is a binary classification problem, where the target variable is the venomosity of the mushroom.

To achieve our goal, we will follow the standard machine learning pipeline, which consists of analyzing the data, preprocessing it to ensure higher accuracy, and, finally, training and comparing the models.

## Table of Contents

1. [Coding environment](#coding-environment)
    1. [Importing the Libraries](#importing-the-libraries)
    2. [Loading the Dataset](#loading-the-dataset)
2. [Initial Data Exploration](#initial-data-exploration)
3. [Data Cleaning](#data-cleaning)
    1. [Exploring the Dataset](#exploring-the-dataset)
    2. [Removing Duplicates](#removing-duplicates)
    3. [Filling in Missing Values](#filling-in-missing-values)
    4. [Removing Outliers](#remove-outliers)
    5. [Encoding Qualitative Data](#encoding-qualitative-data)
4. [Training the Models](#training-the-models)

## Coding environment

### Importing the libraries

Due to its extensive machine learning ecosystem, we have opted to use [Python](https://www.python.org/) for this project. As such, before proceeding, it is imperative to prepare our coding environment by importing the libraries we will be working with, namely:

* **[Pandas](https://pandas.pydata.org/)** - For data manipulation and preprocessing.
* **[Scikit-learn](https://scikit-learn.org/stable/)** - For implementing machine learning models and evaluation metrics.
* **[Matplotlib](https://matplotlib.org/)** - For creating graphs, tables, and numerous other data visualization methods.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### Loading the Dataset

Next, we must load the data itself, which is stored in a compressed CSV file. However, there is no need to manually uncompress it, as Pandas handles that automatically.

In [ ]:
df = pd.read_csv('data/train.zip')

## Initial Data Exploration

It is evident that having a solid understanding of the data is paramount to training accurate models. Therefore, having finished the setup, the first step is to **explore** the dataset to uncover its characteristics. To start, below is a small excerpt from our dataset:


In [ ]:
print("First rows from our dataset:")
df.head()

Before proceeding, it is clear that the `id` column offers no significant information, as it simply indicates the index of the corresponding row. As such, we can safely drop it from the dataset.

In [ ]:
df.drop('id', axis=1, inplace=True)

With that out of the way, we should analyze the data types of the remaining columns to get a better picture of the complete dataset.

In [ ]:
df.info()

As such, our dataset contains 21 columns and over 3 million rows. Regarding the columns, only three contain **quantitative** data, whereas the remaining 18 pertain to **qualitative** data. To facilitate further analysis, we will categorize and extract the columns into distinct variables based on their data type.

In [ ]:
target_column = 'class'

# compute the quantitative columns
quantitative_columns = df.select_dtypes(include=['number']).columns
print("Quantitative columns: ", quantitative_columns.tolist())

# compute the qualitative columns EXCEPT the target column
qualitative_columns = df.select_dtypes(include=['object']).columns.drop(target_column)
print("\nQualitative columns: ", qualitative_columns.tolist())

Below is a succint description of each column:

| **Column/Feature**         | **Description**                                                                 |
|:--------------------------:|:-------------------------------------------------------------------------------:|
| `class`                    | Binary label that Indicates if the mushroom is edible or poisonous.             |
| `cap-diameter`             | Width of the cap at its widest point, in centimeters.                           |
| `cap-shape`                | General form of the cap—e.g., flat, conical, bell-shaped, or wavy.              |
| `cap-surface`              | Texture of the cap—smooth, scaly, sticky, or wrinkled.                          |
| `cap-color`                | Cap color, which may change as the mushroom matures.                            |
| `does-bruise-or-bleed`     | Indicates if the mushroom bruises or releases liquid when damaged.              |
| `gill-attachment`          | How gills connect to the stem—free, attached, or descending.                    |
| `gill-spacing`             | Distance between gills—crowded, spaced, or intermediate.                        |
| `gill-color`               | Color of the gills, which may change with age.                                  |
| `stem-height`              | Length of the stem from base to cap, in centimeters.                            |
| `stem-width`               | Thickness of the stem—narrow, medium, or thick, in centimeters.                 |
| `stem-root`                | Shape of the stem’s base—tapered, swollen, or bulbous.                          |
| `stem-surface`             | Stem texture—smooth, fibrous, scaly, or rough.                                  |
| `stem-color`               | Color of the stem, which may be uniform or variable.                            |
| `veil-type`                | Indicates a partial or universal veil.                                          |
| `veil-color`               | Color of the veil, useful for identification.                                   |
| `has-ring`                 | Indicates if a ring (annulus) is present on the stem.                           |
| `ring-type`                | Type of ring—single, double, flaring, or hanging.                               |
| `spore-print-color`        | Color of spores left on a surface; key for identification.                      |
| `habitat`                  | Environment where the mushroom grows—e.g., woods or grasslands.                 |
| `season`                   | Time of year when the mushroom is typically found.                              |


## Data Cleaning

Now that we have a basic understanding of the dataset, we must prepare it for analysis by addressing any inconsistencies, errors, or missing values that could skew our results.

### Removing Duplicates

Firstly, it is important to determine if the dataset contains **duplicate** rows, as those can be safely excluded without affecting the accuracy of our models.

In [ ]:
print("The dataset contains {} duplicates.".format(df.duplicated().sum()))

As the dataset contains no duplicates, that means all rows provide relevant information, so none must be removed.

### Filling in Missing Values

Another key concern has to do with **missing values**, that is, entries that are absent from the dataset. Seeing as these provide no information, it might be sensible to either delete the rows where they appear or replace the missing entries with meaningful values.

The following highlights the amount of missing values per column:

In [ ]:
print("Missing Values per column (%):")
pd.DataFrame({
    'Count': df.isna().sum(),
    'Frequency (%)': 100 * df.isna().mean(),
})

In [ ]:
plt.figure(figsize=(12,12))
plt.title("Visualizing Missing Values")
sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
plt.show()

It is evident that the dataset has an overabundance of missing values, with some columns having over half of its entries missing. Because of this, we will opt to **fill in** the missing values, as removing the rows where they appear would result in a tremendous data loss.


#### Quantitative Data

There are several methods to fill in missing numerical data. However, the most appropriate for each column depends on the **distribution** of its values:

* If the values are symmetrically distributed, it is appropriate to fill the missing entries with the **mean** as it represents the central tendency more accurately.
* If the values are asymmetrically distributed (**skewed**), then the **median** should be used because it is less affected by outliers.

The following depicts the distribution of each quantitative column:

In [ ]:
# compute the skewness
print("Skewness by column:")
print(df[quantitative_columns].skew())

# plot the distribution
plt.figure(figsize=(4 * len(quantitative_columns), 4))

for index, column in enumerate(quantitative_columns):
    plt.subplot(1, len(quantitative_columns), index+1)
    sns.histplot(data=df, x=column, kde=True, bins=20, stat='probability')
    plt.title(f'Distribution of {column}')
    plt.ylabel('Frequency')
    sns.despine()

plt.tight_layout() # adjust subplots to fit into figure area
plt.show()

Considering all quantitative columns are right-skewed, we must fill their missing values with the median.

In [ ]:
for column in quantitative_columns:
    # compute the median of the column's values
    median = df[column].median()

    # fill the missing values with the median
    df[column] = df[column].fillna(median)

#### Qualitative Data

Handling missing values in qualitative data requires imputation strategies that consider the nature of the data, such as using the **mode**, creating a new **category**, or employing more advanced techniques based on relationships within the data.

As became apparent in ..., there are plenty of qualitative columns where over half the entries are missing (`stem-root`, `veil-type`, `veil-color`, etc.), but there are also a few where only a small percentage is absent (`cap-shape`, `cap-color`, `does-bruise-or-bleed`, etc). So, we will take this into account when replacing the missing entries:
* If more than a predetermined percentage of data is missing, we create a new category - `Unspecified` - to group these unspecified values.
* Otherwise, we fill the missing values with the column's mode so as to preserve the distribution as much as possible.

As for the threshold, we believe 1% will help preserve the accuracy.

In [ ]:
def fill_missing_qualitative_data(data: pd.Series, threshold: int) -> pd.Series:
    '''Fills missing qualitative data based on the number of missing values.'''
    missing_values = data.isna().sum() / len(data)
    mode = data.mode()

    return data.fillna('Unspecified' if missing_values > threshold or mode.empty else mode[0])


# replace the missing values with the mean
for column in qualitative_columns:
    df[column] = fill_missing_qualitative_data(df[column], 0.01)

## Exploratory Data Analysis

Upon cleaning the dataset, the next step is to analyze the underlying patterns and relationships within the data.

### Detecting Outliers

Having ensured all dataset entries have a value, it is appropriate to handle any **outliers** to avoid training our models with unrepresentative data.

#### Quantitative Data

To detect outliers in quantitative data, we can start by plotting the box plot of the respective columns as this type of graph is ideal for easily identifying extremes.

In [ ]:
def display_box_plots(df, quantitative_columns):
    '''Plots the distribution of the quantitative columns.'''
    plt.figure(figsize=(4 * len(quantitative_columns), 4))

    for index, column in enumerate(quantitative_columns):
        plt.subplot(1, len(quantitative_columns), index+1)
        sns.boxplot(data=df, x=column)
        plt.title(f'Box plot of {column}')
        sns.despine()

    plt.tight_layout() # adjust subplots to fit into figure area
    plt.show()


# plot the distributions
display_box_plots(df, quantitative_columns)

From the plots above, we can conclude that there are several outliers. However, in order to decide how to deal with them, we need to understand just how many there are. 

In [ ]:
def get_outliers(data: pd.Series, lower_quantile: float) -> pd.Series:
    '''Computes the outliers of a given data column.'''
    Q1 = data.quantile(lower_quantile)
    Q3 = data.quantile(1 - lower_quantile)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return data[(data < lower_bound) | (data > upper_bound)]


# calculate the percentage of outliers for each quantitative column
print("Outliers by column (%):")

for column in quantitative_columns:
    outliers = get_outliers(df[column], 0.25)
    outliers_percentage = 100 * len(outliers) / len(df[column])

    print(f'{column}\t{round(outliers_percentage, 2)}')

As evidenced, with $Q_{0.25}$ and $Q_{0.75}$ as lower and upper bounds, each quantitative column contains fewer than 5% outliers. Therefore, removing the rows where they appear would not incur a severe data loss for individual columns. However, assuming the worst-case scenario of only one outlier per row, we would be losing over 8% of the dataset, which is not ideal.

One possible solution would be to reduce the threshold and obtain fewer outliers. However, considering our quantitative columns are right-skewed, we believe applying a variance-stabilizing transformation should be more fruitful, as it would not only reduce the impact of large values but also normalize the distribution of our columns. With that in mind, we will apply a **logarithmic transformation** to the quantitative columns.

In [ ]:
# apply logarithmic transformation
for column in quantitative_columns:
    df[column] = np.log1p(df[column])

# view the updated box plots
display_box_plots(df, quantitative_columns)

#### Qualitative Data

Even though outliers are commonly associated with quantitative data, they can also occur in qualitative data. In this case, the term refers to categories that are atypically infrequent i.e. that appear significantly less than the others.

To identify these values, we will compute the percentage of each category per column.

In [ ]:
for column in qualitative_columns:
    # compute the frequency of each category
    freqs = pd.DataFrame({
        'Count': df[column].value_counts(),
        'Frequency (%)': df[column].value_counts(normalize=True) * 100
    })

    print(f"\nCategory Frequency in '{column}'")
    print(freqs.head(10))

As depicted, some categories appear very infrequently ($< 0.01\%$). Despite the desire to include all unique categories for greater fidelity, keeping these rare values in the dataset may introduce noise, as they are unlikely to provide meaningful patterns and can instead take focus away from the dominant trends.

An obvious solution to this problem would be to remove the rows containing these categories, which would naturally result in data loss. However, we can take advantage of the fact that we have a category for missing values - `Unspecified` - and use it to encapsulate these atypical categories, thus achieving the desired effect while preserving all rows.

In [ ]:
def replace_infrequent_qualitative_data(data: pd.Series, freq_threshold: float)-> pd.Series:
    '''Replaces rare categorical values with 'Unspecified'.'''
    # compute the frequency of each category
    infrequent = data.value_counts(normalize=True)[lambda x : x * 100 < freq_threshold].index

    # replace the infrequent values
    return data.apply(lambda x: 'Unspecified' if x in infrequent else x)


# count the number of unique categories per column
before_unique_categories = df[qualitative_columns].nunique()

# replace the infrequent values in each column
for column in qualitative_columns:
    df[column] = replace_infrequent_qualitative_data(df[column], 0.01)

# compare the number of unique categories before and after processing
print("Unique Categories per column:")
pd.DataFrame({
    'Before': before_unique_categories,
    'After': df[qualitative_columns].nunique()
})

### Exploring Correlations

Next, we will identify possible relationships between features by computing their correlations with each other and with the target variable. This analysis will deepen our understanding of the dataset and may also help us eliminate highly collinear features, reducing redundancy and improving model performance.

#### Quantitative vs Quantitative

To examine the strength between pairs of quantitative features, we will compute the **Pearson correlation coefficient**, which is a statistical measure that quantifies how two variables move together. It ranges from -1 (perfect negative correlation) to 1 (perfect positive correlation). The results are as follows:

In [ ]:
sns.heatmap(df[quantitative_columns].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

As illustrated, the maximum correlation between quantitative variables pertains to `cap-diameter` and `stem-width` and is $88\%$. So, if the two are similarly related to the target variable, it might be wise to merge them.

#### Quantitative vs Target

Next, we will determine how each quantitative variable relates to the target variable. To that end, we will use the **Point-Biserial Correlation Coefficient**, which is a particular case of Pearson's correlation coefficient designed for comparing continuous variables with a binary variable. Similarly to Pearson's, the coefficient varies from -1 to 1.

In [ ]:
# encode the target variable
encoded_target = df[target_column].map({ 'e': 0, 'p': 1})

# compute the correlations
correlations = {}

for column in quantitative_columns:
    corr_coeff, _ = stats.pointbiserialr(df[column], encoded_target)
    correlations[column] = [corr_coeff] # store as a list to create a DataFrame row

# convert the correlations to a DataFrame
correlation_df = pd.DataFrame(correlations, index=['class']).T

# plot the correlation matrix
sns.heatmap(correlation_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

As denoted, all quantitative features are weakly correlated with the target variable. Nevertheless, both `cap-diameter` and `stem-width` are identically related to the target, which, adding to the fact they are strongly correlated with each other, further corroborates that we should join them.

#### Qualitative vs Target

The final correlations we will compute are between the qualitative features and the target class. To accomplish this, we will use **mosaic plots** to directly visualize how each category in a qualitative column is related to the target.

In [ ]:
def cramers_v(confusion_matrix):
    """
    Calculates Cramer's V statistic for a given confusion matrix.
    confusion_matrix: A 2D numpy array or pandas DataFrame representing the contingency table.
    """
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    min_dim = min(confusion_matrix.shape) - 1
    
    # Handle the case where min_dim is 0 to avoid division by zero
    if min_dim == 0:
        return 0.0
    
    # Calculate Cramer's V, ensuring no division by zero for n
    if n == 0:
        return 0.0
    
    return np.sqrt(chi2 / (n * min_dim))

results = []

for column in qualitative_columns:
    print(f"\nAnalyzing '{column}' vs 'target':")
    
    # Create contingency table
    contingency_table = pd.crosstab(df[column], df[target_column])
    print("Contingency Table:")
    print(contingency_table)
    
    # Perform Chi-squared test
    chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)
    print(f"Chi-squared statistic: {chi2:.4f}")
    print(f"P-value: {p_value:.4f}")
    
    # Calculate Cramer's V
    v_value = cramers_v(contingency_table)
    print(f"Cramer's V: {v_value:.4f}")
    
    results.append({
        'Feature': column,
        'Chi2_Statistic': chi2,
        'P_Value': p_value,
        'Cramers_V': v_value
    })

# Summarize results in a DataFrame
results_df = pd.DataFrame(results).sort_values(by='Cramers_V', ascending=False)
print("\n--- Summary of Qualitative Feature Associations ---")
print(results_df)

# --- 4. Visualize with Stacked Bar Charts ---
print("\n--- Visualizing Associations with Stacked Bar Charts ---")

for col in qualitative_columns:
    plt.figure(figsize=(10, 6))
    
    # Create normalized contingency table for percentages
    # This shows the proportion of 0s and 1s within each category of 'col'
    normalized_contingency = pd.crosstab(df[col], df[target_column], normalize='index')
    
    # Plotting
    normalized_contingency.plot(kind='bar', stacked=True, colormap='viridis', ax=plt.gca())
    
    plt.title(f'Target Distribution by {col} (Normalized)')
    plt.xlabel(col)
    plt.ylabel('Proportion')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Target', labels=['0', '1'])
    plt.tight_layout()
    plt.show()

### Balancing the Dataset

The final exploratory step concerns the distribution of the target variable itself. More specifically, it is crucial to verify if the dataset is **balanced**, that is, whether there is a roughly equal number of edible and poisonous mushroom samples. This is necessary because training models on unbalanced data can cause them to inadvertedly be biased towards the most prevalent class.

The following showcases the class distribution.

In [ ]:
class_counts = df[target_column].value_counts().sort_index()

plt.figure(figsize=(6, 6))
plt.pie(class_counts, labels=["Edible", "Poisonous"], autopct='%1.1f%%', startangle=90)
plt.title('Class Distribution')
plt.show()

Considering the percentage of edible and poisonous mushrooms differs by less than $10\%$, the dataset is reasonably balanced, so there is no need to perform any additional steps to alter the target value distribution.

## Training the Models

### Encoding Qualitative Data

As most machine learning algorithms and statistical models can only process numerical input, the final preprocessing step is transforming qualitative data into quantitative data.

There are several encoding techniques that achieve this, but we will go with **label encoding**, which merely consists of assigning unique integer values to distinct categories.

In [ ]:
# initialize the label encoder
encoder = LabelEncoder()

# apply Label Encoding to all qualitative data
for column in df[qualitative_columns]:
    df[column] = encoder.fit_transform(df[column])

In [ ]:
# Features (X) and target (y)
y = df['class']
X = df.drop('class', axis=1)

In [ ]:
def evaluate_models(models, X, y, scoring_metrics: list[str], cv: int):
    '''
    Evaluates a list of models using cross-validation and returns a Pandas DataFrame
    with the mean scores for each model.
    '''

    # initialize the results
    results = {}

    for model in models:
        model_name = type(model).__name__

        # perform cross-validation
        scores = cross_validate(model, X, y, scoring=scoring_metrics, cv=cv)

        # calculate the mean of each scoring metric
        results[model_name] = {
            metric: np.mean(scores[f"test_{metric}"]) for metric in scoring_metrics
        }

    # convert the results to a Pandas data frame
    return pd.DataFrame(results).T

In [ ]:
# initialize the model
models = [
    DecisionTreeClassifier(random_state=21),
]

# define the scoring metrics
scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# define the scoring metrics
evaluate_models(models, X, y, scoring_metrics, 3)